# Load Data

In [1]:
from pyearth import Earth
from DLtools.Data_preprocess import series_to_supervised,load_data
from DLtools.Data import load_data,instant_data,check_specific_col

import numpy as np
import pandas as pd

In [2]:
# loaddata = load_data()

# df_d = loaddata.daily()
# df_h = loaddata.hourly()
###################################
loading = instant_data()
df_d = loading.daily_instant()
df_h = loading.hourly_instant()



In [42]:
data = df_h['2013-01-01':'2017-12-31'].interpolate(limit=360).apply(lambda x: x.fillna(x.mean()),axis=0).astype('float32')
data.shape

(43824, 269)

In [33]:
TARGET = 'CPY015_wl'

## Period 2013-2017 ##
data = df_h['2013-01-01':'2017-12-31'].interpolate(limit=360).apply(lambda x: x.fillna(x.mean()),axis=0).astype('float32')

# ## Shift target station to future 
n_out = 24
data[TARGET]=data[TARGET].shift(-n_out)
data = data.astype('float32').dropna()

X = data.drop([TARGET],axis=1)
xlabels = list(X.columns)


# # # REPLACE NAN WITH 0
# # X = X.fillna(0).values
# # y = np.log(data[TARGET].fillna(0).values)
# # where_are_NaNs = np.isnan(y)
# # y[where_are_NaNs] = 0
X=X.values
y=data[TARGET].values

# MARS feature selection (from all stations)

In [34]:
#Fit an Earth model
criteria = ('rss', 'gcv', 'nb_subsets')
model = Earth(enable_pruning = True,
            #   max_degree=3,
            #   max_terms=10,
              minspan_alpha=.5,
              feature_importance_type=criteria,
              verbose=True)
model.fit(X,y,xlabels=xlabels)


Beginning forward pass
---------------------------------------------------------------
iter  parent  var  knot  mse       terms  gcv    rsq    grsq   
---------------------------------------------------------------
0     -       -    -     0.386477  1      0.386  0.000  0.000  
1     0       29   24836  0.212777  3      0.213  0.449  0.449  
2     0       15   21853  0.131452  5      0.132  0.660  0.660  
3     0       11   20933  0.106335  7      0.106  0.725  0.725  
4     0       15   6704  0.098503  9      0.099  0.745  0.745  
5     0       15   24357  0.095587  11     0.096  0.753  0.752  
6     0       14   16510  0.093184  13     0.093  0.759  0.759  
7     0       53   2497  0.091425  15     0.092  0.763  0.763  
8     0       56   38957  0.090218  17     0.090  0.767  0.766  
9     0       52   31493  0.088980  19     0.089  0.770  0.769  
10    0       53   19429  0.088086  21     0.088  0.772  0.772  
11    0       15   7130  0.087544  23     0.088  0.773  0.773  
12    0  

Earth(feature_importance_type=('rss', 'gcv', 'nb_subsets'), minspan_alpha=0.5,
      verbose=True)

In [5]:
# #Print the model
# print(model.trace())
# print(model.summary())

* Feature Importance

In [25]:
print(model.summary_feature_importances(sort_by='rss')[:2000])

                  rss    gcv    nb_subsets
GLF001_wl         0.76   0.76   0.11          
CPY014_wl         0.19   0.19   0.16          
PAS008_wl         0.02   0.02   0.11          
CPY011_wl         0.01   0.01   0.05          
PAS009_wl         0.00   0.00   0.11          
CPY001_wl         0.00   0.00   0.05          
PAS007_wl         0.00   0.00   0.11          
CPY012_wl         0.00   0.00   0.05          
CPY009_wl         0.00   0.00   0.05          
DNP022_wl         0.00   0.00   0.11          
CPY006_wl         0.00   0.00   0.11          
DNP015_temp       0.00   0.00   0.00          
DNP017_temp       0.00   0.00   0.00          
DNP013_temp       0.00   0.00   0.00          
DNP019_temp       0.00   0.00   0.00          
DNP020_temp       0.00   0.00   0.00          
DNP022_temp       0.00   0.00   0.00          
DNP023_temp       0.00   0.00   0.00          
DNP011_temp       0.00   0.00   0.00          
DNP010_temp       0.00   0.00   0.00          
DNP008_temp      

In [35]:
def toDF(rank):
    name,rss,gcv,nb_sub= list(),list(),list(),list()
    for i in range(len(rank)):
        if i%4==0:
            name.append(rank[i])
        elif i%4==1:
            rss.append(rank[i])
        elif i%4==2:
            gcv.append(rank[i])
        elif i%4==3:
            nb_sub.append(rank[i])
    data = {'feature':name,
    'rss':rss,
    'gcv':gcv,
    'nb_subset':nb_sub}
    score = pd.DataFrame(data)
    return score



In [36]:
nbsub = model.summary_feature_importances(sort_by='nb_subsets')[:2000].split()[3:83]
gcv = model.summary_feature_importances(sort_by='gcv')[:2000].split()[3:83]
rss = model.summary_feature_importances(sort_by='rss')[:2000].split()[3:83]


In [37]:
rss = toDF(rss)
gcv = toDF(gcv)
nbsub = toDF(nbsub)

In [38]:
#From top 20 rss,gcv,nbsub
top20 = pd.concat([rss,gcv,nbsub],ignore_index=True).drop_duplicates('feature')
top20

,feature,rss,gcv,nb_subset
0,GLF001_wl,0.56,0.56,0.06
1,CPY014_wl,0.34,0.34,0.26
2,CPY010_wl,0.07,0.07,0.06
3,PAS008_wl,0.01,0.01,0.10
4,CPY006_wl,0.01,0.01,0.06
5,CPY013_wl,0.00,0.00,0.06
6,PAS009_wl,0.00,0.00,0.06
7,PIN003_wl,0.00,0.00,0.06
8,PAS007_wl,0.00,0.00,0.06
9,DNP022_wl,0.00,0.00,0.06


In [39]:
top20.to_csv('featurelist_MAR_hour24.csv',index=False)